In [6]:
import SolarStormwatchIIAnalysis as ssw
import simplejson as json

In [49]:
classifications = ssw.import_classifications(latest=True)

subjects = ssw.import_subjects(active=True)

Error, invalid path, check config: E:\STEREO\ares.nrl.navy.mil\lz
Error, invalid path, check config: E:\STEREO\ares.nrl.navy.mil\lz


In [81]:
reload(ssw)

classifications = ssw.import_classifications(latest=True)

subjects = ssw.import_subjects(active=True)

for ids, sub in subjects.iterrows():
    print sub['metadata']
    break

for ids, cls in classifications.iterrows():
    print cls['annotations']['task']
    break

Error, invalid path, check config: E:\STEREO\ares.nrl.navy.mil\lz
Error, invalid path, check config: E:\STEREO\ares.nrl.navy.mil\lz
{'img_type': 'norm', 'subject_id': '0', 'subject_name': 'ssw_009_swpc_638_sta_norm_20121005T092901_20121005T104901', 'asset_2': 'ssw_009_swpc_638_sta_norm_20121005_104901.jpg', 'asset_0': 'ssw_009_swpc_638_sta_norm_20121005_092901.jpg', 'asset_1': 'ssw_009_swpc_638_sta_norm_20121005_100901.jpg'}
T1


In [18]:
subjects = ssw.import_subjects(active=True)

event_ids = []

for ids, sub in subjects.iterrows():
    meta = json.loads(sub['metadata'])
    name = meta['subject_name']
    ssw_code = name.split('_')
    ssw_code = "_".join(ssw_code[0:2])
    event_ids.append(ssw_code)

# Get unique event id's
event_ids = list(set(event_ids))

event_tree = dict()
for e in event_ids:
            event_tree[e] = {'sta':{'norm':[], 'diff':[]}, 'stb':{'norm':[], 'diff':[]}}

# Now populate the lists in the subject sets with the individual subsets 
for ids, sub in subjects.iterrows():
    meta = json.loads(sub['metadata'])
    name = meta['subject_name']
    name = name.split('_')
    ssw_code = "_".join(name[0:2])
    craft  = name[4]
    im_type = name[5]
    
    event_tree[ssw_code][craft][im_type].append(sub['subject_id'])

{'ssw_009': {'sta': {'diff': [], 'norm': []}, 'stb': {'diff': [], 'norm': []}}}
{'ssw_009': {'sta': {'diff': [3215323L, 3215327L, 3215332L, 3215339L, 3215347L, 3215354L, 3215360L, 3215366L, 3215372L], 'norm': [3215239L, 3215249L, 3215259L, 3215267L, 3215276L, 3215287L, 3215296L, 3215305L, 3215314L]}, 'stb': {'diff': [3215672L, 3215679L, 3215686L, 3215693L, 3215699L, 3215704L, 3215711L, 3215718L, 3215726L], 'norm': [3215595L, 3215606L, 3215617L, 3215626L, 3215636L, 3215644L, 3215645L, 3215650L, 3215661L]}}}


In [38]:
# Make a dictionary with details of which subjects and subject sets belong to which 
# Get subject sets belonging to each event
event_dict = dict()
for e in event_ids:
    subject_sets = []
    for ids, sub in subjects.iterrows():
        meta = json.loads(sub['metadata'])
        name = meta['subject_name']
        ssw_code = name.split('_')
        ssw_code = "_".join(ssw_code[0:2])
        if ssw_code == e:
            subject_sets.append(sub['subject_set_id'])


In [ ]:
# Loop through the unique subject sets
    for set_id in subjects['subject_set_id'].unique():
        # Get all subjects in this set
        subject_subset = get_df_subset(subjects, 'subject_set_id', set_id)

        # Get info on the files in this set.
        set_files = get_subject_set_files(subject_subset, [set_id])

        # Create a group for this set
        grp_set = hdf.create_group(str(set_id))
        grp_set.attrs['craft'] = set_files[set_id]['craft']

        # Loop over the subjects in this set
        for ids, sub in subject_subset.iterrows():
            # Get the meta information of assets in the subject
            info = json.loads(sub['metadata'])

            # Find classifications for this subject
            clas_subset = get_df_subset(classifications, 'subject_id', sub['subject_id'])

            # Loop over the assets in this subject
            for asset_id in range(3):
                key = "asset_{}".format(asset_id)
                # Form a timestring for the hdf5 branch name
                file_parts = os.path.splitext(info[key])[0].split('_')
                timestr = "_".join(file_parts[6:8])
                timestr = pd.datetime.strptime(timestr, '%Y%m%d_%H%M%S').isoformat()

                # Make a branch for this image type / asset time.
                branch = "/".join([info['img_type'], timestr])
                print branch
                grp_asset = grp_set.create_group(branch)

                # Loop over the classifications to pull out the annotations for this asset.
                all_x_pix = []
                all_y_pix = []
                all_user = []
                # Counter for the number of submissions on this asset
                submission_count = 0
                for idc, cla in clas_subset.iterrows():

                    # Get the annotation
                    cla_anno = json.loads(cla['annotations'])[0]

                    # Loop through values and find value with frame matching asset index
                    for val in cla_anno['value']:
                        # Does this frame match the asset of interest?
                        if val['frame'] == asset_id:
                            # Get the data for this user classification
                            user_x_pix = [int(p['x']) for p in val['points']]
                            user_y_pix = [int(p['y']) for p in val['points']]
                            user_id = cla['user_id']
                            # Set up a group for this user.
                            branch = "users/user_{}".format(submission_count)
                            print "---->" + branch
                            grp_user = grp_asset.create_group(branch)
                            # Assign the user data to their group
                            grp_user.create_dataset('x_pix', data=np.array(user_x_pix, dtype=int))
                            grp_user.create_dataset('y_pix', data=np.array(user_y_pix, dtype=int))
                            grp_user.create_dataset('id', data=user_id)
                            # Dump users data into collection of annotations.
                            all_x_pix.extend(user_x_pix)
                            all_y_pix.extend(user_y_pix)
                            all_user.append(user_id)
                            submission_count += 1

                # Now add on the collected annotation to the assets group.
                grp_asset.create_dataset('x_pix', data=np.array(all_x_pix, dtype=int))
                grp_asset.create_dataset('y_pix', data=np.array(all_y_pix, dtype=int))
                grp_asset.create_dataset('user_list', data=np.array(all_user, dtype=int))